# Lecture 19: Probability 3

In [ ]:
import numpy as np
import sympy as sp
import scipy.integrate
sp.init_printing()
##################################################
##### Matplotlib boilerplate for consistency #####
##################################################
from ipywidgets import interact
from ipywidgets import FloatSlider
from matplotlib import pyplot as plt
import math

%matplotlib inline

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg')

global_fig_width = 10
global_fig_height = global_fig_width / 1.61803399
font_size = 12

plt.rcParams['axes.axisbelow'] = True
plt.rcParams['axes.edgecolor'] = '0.8'
plt.rcParams['axes.grid'] = True
plt.rcParams['axes.labelpad'] = 8
plt.rcParams['axes.linewidth'] = 2
plt.rcParams['axes.titlepad'] = 16.0
plt.rcParams['axes.titlesize'] = font_size * 1.4
plt.rcParams['figure.figsize'] = (global_fig_width, global_fig_height)
plt.rcParams['font.sans-serif'] = ['Computer Modern Sans Serif', 'DejaVu Sans', 'sans-serif']
plt.rcParams['font.size'] = font_size
plt.rcParams['grid.color'] = '0.8'
plt.rcParams['grid.linestyle'] = 'dashed'
plt.rcParams['grid.linewidth'] = 2
plt.rcParams['lines.dash_capstyle'] = 'round'
plt.rcParams['lines.dashed_pattern'] = [1, 4]
plt.rcParams['xtick.labelsize'] = font_size
plt.rcParams['xtick.major.pad'] = 4
plt.rcParams['xtick.major.size'] = 0
plt.rcParams['ytick.labelsize'] = font_size
plt.rcParams['ytick.major.pad'] = 4
plt.rcParams['ytick.major.size'] = 0
##################################################

## Standard distributions

- Standard distributions, such as the Normal distribution, give succinct 
  descriptions of commonly occurring kinds of variation, and hence have well 
  known properties.
- But remember that any function summing/integrating to 1 can also describe a 
  distribution.

<!--Presentation notes: get the audience to think of examples for each 
distribution, and display histograms/pdfs in Matlab for different parameter 
values.  Make sure to derive expectation & variance in each case.
% TODO: Write a Matlab script for the above!-->

## Bernoulli

An outcome can be either $\;1\;$ ("success") or $\;0\;$ ("failure") with probabilities $\;\theta\;$ and $\;(1-\theta)\;$ respectively.

- The probability of observing a sequence of $\;n\;$ samples with $\;k\;$ 1s is $\;\theta^k(1-\theta)^{n-k}$.
- The expectation is $\;\theta\;$ and the variance is $\;\theta(1-\theta)$.

We write $\;X\sim\text{Bernoulli}(\theta)\;$ to say that $\;X\;$ has a Bernoulli distribution with parameter $\;\theta$.

## Binomial: $\;X\sim B(n,\theta)$

Often we don't care about the order in which "successes" or "failures" happen, just how many there are of each.

- The probability of $\;k\;$ successes in $\;n\;$ Bernoulli trials is given by the **Binomial** distribution.
- Each order has the same probability (given by the Bernoulli formula above), 
  and the number of possible orderings is $\;_nC_k,\;$ so

$$ P(X=k|n,\theta) = {n\choose k}\theta^k(1-\theta)^{n-k} = \frac{n!}{(n-k)!k!}\theta^k(1-\theta)^{n-k} $$

- The expectation is $\;n\theta\;$ and the variance is $\;n\theta(1-\theta)$.

In [ ]:
from scipy.stats import binom
n = 1000
d = binom(100, 0.5)
b = np.arange(30, 70)
plt.title('Binomial distribution, n=100, p=0.5')
plt.hist(d.rvs(n), bins=b, density=True, alpha=0.5,label='sampled')
plt.plot(b+0.5, d.pmf(b), label='Ideal');

## Geometric: $\;X\sim\text{Geom}(\theta)$

- The probability of success in a Bernoulli trial is not influenced by (is 
  independent of) previous trials; this is known as the **memory-less** property.

- The number of trials between subsequent successes, or equivalently the 
  probability that the first success is on the $\;k^{\rm{th}}$ trial, has the 
  **Geometric** distribution.

$$ P(X=k|\theta) = \theta(1-\theta)^{k-1} $$

- The expectation is $\;1/\theta\;$ and the variance is $\;(1-\theta)/\theta^2$.

In [ ]:
from scipy.stats import geom
n = 1000
d = geom(0.3, loc=-1)
b = np.arange(0, 30)
plt.title('Geometric distribution (p=0.3)')
plt.hist(d.rvs(n), bins=b, density=True, alpha=0.5,label='sampled')
plt.plot(b+0.5, d.pmf(b), label='Ideal');

## Poisson: $\;X\sim\text{Pois}(\mu)$

- This distribution models the number of times an event occurs in a fixed interval of time (and/or space) if these events occur with a known fixed average rate $\;\mu\;$ and independently of the time since the last event.

- It is is often used to model "rare" events, e.g. the number of deaths per year in a given age group.

- It can be derived as the limit of the Binomial distribution as the number of trials increases $\;(n\to\infty)\;$ but the expectation $\;\mu\;$ is fixed (so $\;\theta\to 0$).

$$ P(X=k|\mu) = \frac{e^{-\mu} \mu^k}{k!} $$

- The expectation is $\;\mu\;$ and the variance is also $\;\mu$.

<!--% TODO: Matlab script should compare Poisson(4) with Binomial(200,0.02) 
histograms-->

In [ ]:
from scipy.stats import poisson
n = 1000
mu = 40
d = poisson(mu)
b = np.arange(0, 3*mu)
plt.title('Poisson distribution ($\mu=%d$)'%mu)
plt.hist(d.rvs(n), bins=b, density=True, alpha=0.5,label='sampled')
plt.plot(b+0.5, d.pmf(b), label='Ideal')
plt.legend();

In [ ]:
from scipy.stats import binom, poisson
N, p = 1600, 0.0025
db = binom(N, p)
dp = poisson(N*p)
n = 1000
b = np.arange(0, 15)
plt.title('Comparing binomial and Poisson')
plt.hist(db.rvs(n), bins=b, density=True, alpha=0.5,label='Binomial (' + str(N) + ', ' + str(p) + ')')
plt.hist(dp.rvs(n), bins=b, density=True, alpha=0.5,label='Poisson (' + str(N*p) + ')')
plt.legend();

## Uniform: $\;X\sim U(a,b)$

- Here the random variable has the same probability across the whole domain.
- It is often used as a prior in *Bayesian inference* to represent no special *a priori* knowledge.

- The pdf and cdf for a uniform distribution between $\;a\;$ and $\;b\;$ are:
$$
f(x|a,b) = \begin{cases}
    \frac{1}{b-a} & \text{if } a < x \leq b \\
    0             & \text{otherwise}
\end{cases},
\qquad
F(x) = \begin{cases}
    \frac{x-a}{b-a} & \text{if } a < x \leq b \\
    0               & \text{if } x \leq a \\
    1               & \text{if } x > b
\end{cases}
$$

- The expectation is $\;\displaystyle\frac{1}{2}(a+b)\;$ and the variance is $\;\displaystyle\frac{1}{12}(b-a)^2$.

In [ ]:
from scipy.stats import uniform
n = 1000
lower, upper = 5, 15
d = uniform(lower, upper - lower)
x = np.linspace(lower, upper)
plt.xlim(lower, upper)
plt.ylim(0, 1.5 / (upper - lower))
plt.title('Uniform distribution (5, 15)')
plt.hist(d.rvs(n), bins=10, density=True, alpha=0.5,label='sampled')
plt.plot(x, d.pdf(x), label='Ideal')
plt.legend();

## Exponential: $\;X\sim\text{Exp}(\lambda)$

- This describes the time between events in a Poisson process, i.e. a process in which events occur continuously and independently at a constant average rate $\;\lambda\;$ (and multiple events cannot happen at the same instant).

- It can also be used for the distance between events in space under the 
  corresponding assumptions.

- It is the continuous analogue of the Geometric distribution, and has the corresponding memory-less property. For continuous distributions, this is $\;f(x+t|X>t)=f(x)$.

The pdf and cdf are:

$$
f(x|\lambda) = \begin{cases}
    \lambda e^{-\lambda x} & \text{if } x \geq 0 \\
    0                   & \text{otherwise}
\end{cases},
\qquad
F(x) = \begin{cases}
    1-e^{-\lambda x} & \text{if } x \geq 0 \\
    0               & \text{if } x < 0
\end{cases}
$$

- The expectation is $\;1/\lambda\;$ and the variance is $\;1/\lambda^2$.

In [ ]:
from scipy.stats import expon
n = 1000
mu = 5
d = expon(loc=0, scale=mu)
x = np.linspace(0, 36, 1000)
plt.xlim(0, 36)
plt.title('Exponential distribution (mu=5)')
plt.hist(d.rvs(n), bins=18, density=True, alpha=0.5,label='sampled')
plt.plot(x, d.pdf(x), label='Ideal')
plt.legend();

## Gamma: $\;X\sim\text{Gamma}(\alpha, \beta)$

- The gamma distribution arises as the sum of iid exponentially distributed 
  random variables
- if $\;X_i\sim\text{Exp}(\lambda)\;$ and $\;S = X_1 + X_2 + \ldots + X_n\;$ then 
  $\;S\sim\text{Gamma}(n,\lambda)$.

The pdf and cdf are:

$$
f(x|\alpha,\beta) = \begin{cases}
    \frac{\beta^\alpha}{\Gamma(\alpha)}x^{\alpha-1}e^{-\beta x} & \text{if } x \geq 0 \\
    0                                                    & \text{otherwise}
\end{cases},
\quad
F(x) = \begin{cases}
    \frac{\gamma(\alpha,\beta x)}{\Gamma(\alpha)} & \text{if } x \geq 0 \\
    0                                             & \text{if } x < 0
\end{cases}
$$

where $\;\alpha = n\;$, $\;\beta = \lambda\;$, $\;\Gamma\;$ is the gamma function (an extension of the factorial function), and $\;\gamma\;$ is the lower incomplete gamma function.

- The expectation is $\;\alpha/\beta\;$ and the variance is $\;\alpha/\beta^2$.

In [ ]:
from scipy.stats import gamma
n = 1000
d = gamma(a=2, scale=2)
x = np.linspace(0, 20, 1000)
plt.xlim(0, 20)
plt.title('Gamma distribution (a=2, b=2)')
plt.hist(d.rvs(n), bins=20, density=True, alpha=0.5,label='sampled')
plt.plot(x, d.pdf(x), label='Ideal')
plt.legend();

In [ ]:
from scipy.stats import expon, gamma
n = 1000
mu = 5
de = expon(loc=0, scale=mu)
dg = gamma(a=1, scale=mu)
x = np.linspace(0, 36, 1000)
b = np.arange(0, 36, 2)
plt.xlim(0, 36)
plt.title('Comparing exponential and gamma')
plt.hist(de.rvs(n), bins=b, density=True, alpha=0.5,label='Exponential (mu=5)')
plt.hist(dg.rvs(n), bins=b, density=True, alpha=0.5,label='Gamma (a=1, b=5)')
plt.plot(x, de.pdf(x), '--', label='Ideal')
plt.legend();

## Normal: $X\sim N(\mu, \sigma^2)$

- This is a very commonly arising distribution in the natural world
- it is the limiting case of the Binomial and Poisson distributions with large 
  means.
- This ubiquity is explained by the **central limit theorem**, which relates 
  the normal to most distributions through the mean of a large number of 
  samples.

<!--% TODO: Show Normal & Poisson on same plot in Matlab script-->


$$ f(x|\mu,\sigma^2) = \frac{1}{\sigma\sqrt{2\pi}} e^{-((x-\mu)^2/2\sigma^2)} $$

In [ ]:
from scipy.stats import norm
n = 1000
d = norm(5, 2)
x = np.linspace(-4, 14, 1000)
b = np.arange(-4, 14, 0.5)
plt.xlim(-4, 14)
plt.hist(d.rvs(n), bins=b, density=True, alpha=0.5,label='sampled')
plt.plot(x, d.pdf(x), label='Ideal')
plt.legend();

In [ ]:
from scipy.stats import norm, poisson
n = 1000
dp = poisson(100)
dn = norm(100, np.sqrt(100))
plt.xlim(60, 140)
plt.title('Comparing normal and Poisson distribution')
plt.hist(dp.rvs(n), bins=np.arange(60, 145, 5), density=True, alpha=0.5,label='Poisson (100)')
plt.hist(dn.rvs(n), bins=np.arange(62.5, 142.5, 5), density=True, alpha=0.5,label='Normal (100, sqrt(100))')
plt.legend();

## Cautionary Python 

The parameterisations of the distributions in Python do not always match those 
above, so it is wise to check the documentation for each distribution.